In [1]:
import sys
import os
# This code enables using of "src.data" imports in vs code (when you're launching it directly from notebooks directory)
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.append(project_root)

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import transformers
from src.data.classification import SST2Dataset

torch.manual_seed(42)

In [2]:
# Loading model weights
qconf = transformers.BitsAndBytesConfig(load_in_8bit=True)

model_name = "AnatoliiPotapov/T-lite-instruct-0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto",
    torch_dtype="auto",
    quantization_config=qconf,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
#eval_prompt = "Classify the sentiment as positive or negative"

test_sst2_ds = SST2Dataset(
    #prompt=eval_prompt,
    tokenizer=tokenizer,
    data_path="../../data/sst-2/test-00000-of-00001.parquet",
    config_path="../../data/",
    device=model.device
)

In [4]:
test_sst2_ds.prompt

'Please perform Sentiment Classification task\n\nAnswer using the label from [negative, positive].\nGenerate the final answer bracketed with <ans> and </ans>.\n\nThe input:\n<INPUT>\n\nResponse:\n'

In [5]:
# getting first data sample

input_ids, attention_mask, label = next(iter(test_sst2_ds))
print(input_ids.shape, attention_mask.shape, label.shape)

torch.Size([99]) torch.Size([99]) torch.Size([])


In [6]:
# terminators were taken from hf model page (t-lite 0.1)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# generating answer for our sample 
# unsqueeze(0) - to make to necessary shape (when using DataLoader it'll be done automatically)
outputs = model.generate(
    input_ids=input_ids.unsqueeze(0),
    attention_mask = attention_mask.unsqueeze(0),
    max_new_tokens=50,
    eos_token_id=terminators,
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/nfs/home/edyagin/.virtualenvs/prompt_optimization/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [7]:
# decoding the answer

ans = tokenizer.decode(outputs[0], skip_special_tokens=True)
ans

'Please perform Sentiment Classification task\n\nAnswer using the label from [negative, positive].\nGenerate the final answer bracketed with <ans> and </ans>.\n\nThe input:\nno movement, no yuks, not much of anything.\n\nResponse:\n<ans>negative</ans>'

In [18]:
len(input_ids), len(outputs[0])
tokenizer.decode(outputs[0][len(input_ids):], skip_special_tokens=True)

'<ans>negative</ans>'

In [8]:
tokenizer.decode(input_ids)

'<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>Please perform Sentiment Classification task\n\nAnswer using the label from [negative, positive].\nGenerate the final answer bracketed with <ans> and </ans>.\n\nThe input:\nno movement, no yuks, not much of anything.\n\nResponse:\n'

In [9]:
def _extract_label_id_from_answer(answer: str, label2id: dict[str, int]) -> torch.Tensor:
        
        ANS_BEGINNING_TAG = "<ans>"
        ANS_BEGINNING_TAG = "<ans>"
        ANS_ENDING_TAG = "</ans>"

        FORMAT_MISMATCH_LABEL = -1
        start_idx = answer.rfind(ANS_BEGINNING_TAG)

        if start_idx == -1:
            return torch.tensor(FORMAT_MISMATCH_LABEL)

        content_start = start_idx + len(ANS_BEGINNING_TAG)
        end_idx = answer.find(ANS_ENDING_TAG, content_start)

        if end_idx == -1:
            return torch.tensor(FORMAT_MISMATCH_LABEL)

        label = answer[content_start:end_idx]

        return torch.tensor(label2id.get(label, FORMAT_MISMATCH_LABEL))

In [10]:
answers = tokenizer.decode(outputs[0], skip_special_tokens=True)
answers

'Please perform Sentiment Classification task\n\nAnswer using the label from [negative, positive].\nGenerate the final answer bracketed with <ans> and </ans>.\n\nThe input:\nno movement, no yuks, not much of anything.\n\nResponse:\n<ans>negative</ans>'

In [15]:
test_sst2_ds.prompt

'Please perform Sentiment Classification task\n\nAnswer using the label from [negative, positive].\nGenerate the final answer bracketed with <ans> and </ans>.\n\nThe input:\n<INPUT>\n\nResponse:\n'

In [14]:
s = "Response: <hi>"
s.rfind("<ans>")
answers.find(test_sst2_ds.prompt)

-1

In [21]:
def importOrReload(module_name, *names):
    import sys
    from importlib import reload
    
    if module_name in sys.modules:
        reload(sys.modules[module_name])
    else:
        __import__(module_name, fromlist=names)

    for name in names:
        globals()[name] = getattr(sys.modules[module_name], name)

# use instead of: from dfly_parser import parseMessages
importOrReload("src.evaluation.evaluator", "TextClassificationEvaluator")

In [1]:
from src.evaluation.evaluator import TextClassificationEvaluator

model_generate_params = {
    "max_new_tokens": 50,
    "eos_token_id": terminators
}

evaluator = TextClassificationEvaluator()
macro_f1 = evaluator.evaluate(
    model=model, 
    tokenizer=tokenizer,
    eval_ds=test_sst2_ds,
    batch_size=64,
)
macro_f1

ImportError: cannot import name 'TextClassificationEvaluator' from 'src.evaluation.evaluator' (/nfs/home/edyagin/CoolPrompt/src/evaluation/evaluator.py)